In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

data_dir = './' #Change to your directory here

def load_data(data_dir):
    train = pd.read_json(data_dir + 'train.json')
    print("Train data loaded")
    test = pd.read_json(data_dir + 'test.json')
    print("Test data loaded")
    #Fill 'na' angles with mode
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    test.inc_angle = test.inc_angle.replace('na', 0)
    test.inc_angle = test.inc_angle.astype(float).fillna(0.0)
    return train, test

train, test = load_data(data_dir)
print("done")

def color_composite(data):
    w,h = 75,75
    rgb_arrays = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2

        r = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        g = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        b = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))

        rgb = np.dstack((r, g, b))
        #Add in to resize for resnet50 use 197 x 197
        rgb = cv2.resize(rgb, (w,h)).astype(np.float32)
        rgb_arrays.append(rgb)
    return np.array(rgb_arrays)

rgb_train = color_composite(train)
rgb_test = color_composite(test)
print("RGBs Done")

y_train = np.array(train['is_iceberg'])

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(rgb_train, y_train, random_state=420, train_size=0.75)

print("X_train:",X_train.shape)
print("X_valid:",X_valid.shape)

Train data loaded
Test data loaded
done
RGBs Done
X_train: (1203, 75, 75, 3)
X_valid: (401, 75, 75, 3)


C:\Users\Damygame\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPool2D
from keras.layers import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization 
from keras.utils import np_utils
from keras.datasets import mnist
import keras

#Create the model
#model = simple_cnn()
#base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(197,197,3))
#x = base_model.output
#x = GlobalMaxPooling2D()(x)
#x = Dense(1024, activation='relu')(x)
#predictions = Dense(1, activation='sigmoid')(x)

#model = Model(inputs=base_model.input, outputs=predictions)
#for layer in base_model.layers:
 #   layer.trainable = False

def CNN(input_shape=(75, 75, 3)):
    
    def fully_connected(model, n_units, dropout_rate=0):
        model.add(Dense(n_units, kernel_initializer='he_uniform'))
        model.add(PReLU())
        model.add(Dropout(dropout_rate))
        return model
    
    def post_conv(model, dropout_rate=0):
        model.add(BatchNormalization())
        model.add(PReLU())
        model.add(Dropout(dropout_rate))
        model.add(MaxPooling2D(pool_size=(2,2)))
        return model

    def convolutional_layer(model, filters, kernel_size, padding = 'valid', input_shape=None, dropout_rate=0):
        if input_shape is None:
            model.add(Conv2D(filters, kernel_size=kernel_size, padding=padding, kernel_initializer='he_uniform'))
        else:
            model.add(Conv2D(filters, kernel_size=kernel_size, padding=padding, 
                             kernel_initializer='he_uniform', input_shape=input_shape))
        return post_conv(model, dropout_rate)
    
    model = Sequential()
    
    model = convolutional_layer(model, 32, kernel_size=(3,3), dropout_rate=0.2, input_shape=input_shape)
    model = convolutional_layer(model, 64, kernel_size=(3,3), dropout_rate=0.2)
    model = convolutional_layer(model, 128, kernel_size=(3,3), dropout_rate=0.2)  
    #model = convolutional_layer(model, 256, kernel_size=(3,3), dropout_rate=0)    


    model.add(Flatten())
    model = fully_connected(model, 256, dropout_rate=0.3)
    model = fully_connected(model, 128, dropout_rate=0.3)
    model.add(Dense(1, activation='sigmoid'))
    return model


model = CNN()
print(model.summary())


from keras.preprocessing.image import ImageDataGenerator
batch_size = 32
#Lets define the image transormations that we want
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1,
                         rotation_range=40)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=420)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

#Finally create out generator
gen_flow = gen_flow_for_one_input(X_train, y_train)

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
#call backs
earlystop = EarlyStopping(monitor='val_loss', patience=100, verbose=1, min_delta=1e-4, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=40, verbose=1, epsilon=1e-4, mode='min')
model_chk = ModelCheckpoint(monitor='val_loss', filepath=weights_file, save_best_only=True, save_weights_only=True, mode='min')
        
callbacks = [earlystop, reduce_lr_loss, model_chk, TensorBoard(log_dir='../logs')]

model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.002), 
              loss='binary_crossentropy', metrics=['accuracy'])

#fit the model
model.fit_generator(gen_flow, validation_data=(X_valid, y_valid), 
    steps_per_epoch=2**14/batch_size, epochs=500, 
    verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 73, 73, 32)        896       
_________________________________________________________________
batch_normalization_22 (Batc (None, 73, 73, 32)        128       
_________________________________________________________________
p_re_lu_5 (PReLU)            (None, 73, 73, 32)        170528    
_________________________________________________________________
dropout_26 (Dropout)         (None, 73, 73, 32)        0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
batch_normalization_23 (Batc (None, 34, 34, 64)        256       
__________

512/512 [==============================] - 22s 43ms/step - loss: 0.1780 - acc: 0.9259 - val_loss: 0.3828 - val_acc: 0.8853
Epoch 20/500
512/512 [==============================] - 21s 41ms/step - loss: 0.1759 - acc: 0.9275 - val_loss: 0.3249 - val_acc: 0.8653
Epoch 21/500
512/512 [==============================] - 21s 41ms/step - loss: 0.1724 - acc: 0.9315 - val_loss: 0.4156 - val_acc: 0.8579
Epoch 22/500
512/512 [==============================] - 21s 41ms/step - loss: 0.1659 - acc: 0.9308 - val_loss: 0.3101 - val_acc: 0.8778
Epoch 23/500
512/512 [==============================] - 21s 41ms/step - loss: 0.1528 - acc: 0.9381 - val_loss: 0.6140 - val_acc: 0.8005
Epoch 24/500
422/512 [=======================>......] - ETA: 3s - loss: 0.1490 - acc: 0.9393

KeyboardInterrupt: 

In [11]:
# Predict on test data
from keras.models import load_model
model = load_model('model.h5')

print(model.evaluate(X_train, y_train))
print(model.evaluate(X_valid, y_valid))

test_predictions = model.predict(rgb_test)

# Create .csv
pred_df = test[['id']].copy()
pred_df['is_iceberg'] = test_predictions
pred_df.to_csv('model.csv', index = False)

1203/1203 [==============================] - 1s 1ms/step
[0.13342510869824084, 0.94846217798770516]
401/401 [==============================] - 0s 517us/step
[0.23584743628180829, 0.90523690802795331]


In [24]:
model = None